# Olfactory Projection Fibers (OP_3)

In [1]:
from ip.ios import *
from ip.enhancement import *
from ip.binary import *
from ip.graph_nx import *
from ip.swc import *
from ip.utils import *
from skimage.morphology import skeletonize
from skimage.util import img_as_ubyte, img_as_float
from scipy import ndimage

## Read Data

In [2]:
folder_path = "./OlfactoryProjectionFibers/ImageStacks/OP_3"

In [3]:
images = load_tif_stack(folder_path)

In [4]:
imagesToFloat = img_as_float(images)

## Code Process and Execution

In [5]:
filtered = ndimage.median_filter(imagesToFloat, size=3)


In [6]:
#LOG1_0        = img_as_ubyte(ndimage.gaussian_laplace(img_as_ubyte(filtered), [1.0, 2.0, 3.0]))

LOGMEDIAN      = img_as_ubyte(ndimage.gaussian_laplace(filtered, 1.0))

#simple_imshow([blended(images), blended(filtered), blended(LOGMEDIAN)])


In [7]:
#slide_imshow(LOG1_0, True)

In [8]:
threshold3 = mean_threshold(LOGMEDIAN)

In [9]:
binary3 = simple_binary(LOGMEDIAN, threshold3)

In [10]:
#simple_imshow([blended(binary3)])

In [11]:
#slide_imshow(binary3)

In [12]:
segLOG = img_as_ubyte(segment(filtered, ~binary3))

In [13]:
segImages = img_as_ubyte(bilateral_blur(segLOG, 7, 75, 50))

In [14]:
#simple_imshow([blended(segLOG, True),blended(img_as_ubyte(filtered), True),blended(img_as_ubyte(segImages, True))])

In [15]:
#slide_imshow(segImages)

In [16]:
binary4 = simple_binary(segImages, mean_threshold(segImages, 13))
#simple_imshow(blended(binary4, True))

In [17]:
#slide_imshow(binary4)

In [18]:
skel = img_as_ubyte(skeletonize(binary4))
#simple_imshow([blended(skel)])

In [19]:
slide_imshow(skel)

In [20]:
skel[35:38, 182:, :92] = 0
#simple_imshow([blended(skel)])

### Graph generation

In [21]:
graph = Graph(skel)
graph.set_root((37, 181, 93))
graph.create_graph(moving_avg=True, window_moving_avg_sz=3)
root = graph.get_root()
g_root = (93.742,179,38)
print(f"OP_3 GOLD STANDARD ROOT: {g_root}\nTEST ROOT: {root}")

>> Graph created
OP_3 GOLD STANDARD ROOT: (93.742, 179, 38)
TEST ROOT: (37, 181, 93)


In [22]:
mst = graph.apply_dfs_and_label_nodes()

>> Minimum Spanning Tree Generated
>> Minimum Spanning Tree length: 901
>> Depth-First search and labeling complete


In [23]:
graph.save_to_swc(mst,"./Test/OP_3test.swc")

>> SWC saved at: ./Test/OP_3test.swc


True